In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers 
print(etl_helpers.__file__)
from pyspark.sql.functions import collect_list, concat_ws, udf, lit
from pyspark.sql.types import IntegerType

from datetime import datetime, timedelta, date, time
%pip install holidays
%pip install maya
import holidays
import maya

def __getholidaysbyyear(year):        
    ca_holidays = []
    for date, name in sorted(holidays.CA(prov='BC', years=year).items()):
        ca_holidays.append(date.strftime('%Y-%m-%d'))
    return ca_holidays    

def getholidays():
    ca_holidays = []
    currentyear = datetime.today().year
    years = [currentyear - 1, currentyear, currentyear + 1] 
    for year in years:
        ca_holidays.extend(__getholidaysbyyear(year))
    return ca_holidays

def isbusinessday(inpdate, holidays=None):
    _holidays = getholidays() if holidays is None else holidays
    if formatdate(inpdate) not in _holidays and __isweekday(inpdate) == True:
        return True
    return False

def __isweekday( inpdate):
    _inpdate = getdate(inpdate)
    if _inpdate.weekday() < 5:
        return True
    else:   
        return False 
    
def getbusinessdaysbetween( date1, date2=None):
    _holidays = getholidays()
    businessdays = 0
    date2 = date2 if date2 not in (None, '') else gettoday()
    _date1_date_fmt = getdate(date1)
    _date2_date_fmt = getdate(date2)
    __fromdate = _date1_date_fmt if _date1_date_fmt <= _date2_date_fmt else _date2_date_fmt
    __todate = _date2_date_fmt if _date2_date_fmt >= _date1_date_fmt else _date1_date_fmt
    __fromcalcdate =__fromdate
    while getdate(__fromcalcdate).date() < getdate(__todate).date():
        if isbusinessday(__fromcalcdate, _holidays) == True:
            businessdays += 1
        __fromcalcdate =  __fromcalcdate + timedelta(days=1)           
    return businessdays  

def getdate(inputdate):
    print("here3")
    return  datetime.strptime(inputdate, "%Y-%m-%d") if isinstance(inputdate, str) else datetime.combine(inputdate, time.min) if isinstance(inputdate, date) else inputdate   
    # return datetime.strptime(inputdate, "%Y-%m-%d") if isinstance(inputdate, str) else inputdate   


def gettoday(format=None):
    now_pst = maya.parse(maya.now()).datetime(to_timezone='America/Vancouver', naive=False)
    return __formatdate(now_pst, format)

def formatdate( input, format=None):
    _inpdate =  getdate(input)
    return __formatdate(_inpdate, format) 

def __formatdate(_inpdate, format):
    _format = format if format not in (None, '') else "%Y-%m-%d"
    return _inpdate.strftime(_format)

runcycleid = etl_helpers.start_run_cycle("factRequestForDocuments")

try:

    df_existing = spark.sql("SELECT max(TIMESTAMP(runcycleendat)) as runcycleendat from dimruncycle where packagename = 'factRequestForDocuments' and success = 't'")
    df_existing.show()
    maxcreatedate = df_existing.first().runcycleendat
    print(maxcreatedate)
    maxcreatedate_str = maxcreatedate.strftime("%Y-%m-%d %H:%M:%S")

    df_existing = spark.sql(f"SELECT distinct foiministryrequestid, min(foirequest_id) as foirequest_id from foi_mod.foiministryrequests where (created_at > '{maxcreatedate_str}' or TRY_CAST(updated_at AS DATE) > '{maxcreatedate_str}') group by foiministryrequestid")

    df_existing.show()

    comma_list_df = df_existing.select(concat_ws(",", collect_list("foirequest_id")).alias("comma_list"))
    comma_list = comma_list_df.collect()[0]["comma_list"]

    print(comma_list)

    query = f"""MERGE INTO default.factRequestForDocuments dd
        --USING dimRequests r -- commented out since we are adding sourceoftruth column to each fact table
        --ON r.foirequestid = dd.foirequestid
        WHEN MATCHED and dd.foirequestid in ({comma_list}) and sourceoftruth = 'FOIMOD' THEN

        UPDATE 
        SET dd.activeflag = 'N'""";

    comma_list_df = df_existing.select(concat_ws(",", collect_list("foiministryrequestid")).alias("comma_list"))
    comma_list = comma_list_df.collect()[0]["comma_list"]

    print(comma_list)

    query = f"""
        select * from

        (SELECT *
                FROM (
                    SELECT 
                    case when requeststatuslabel = 'closed' then created_at else null end as completeddate,
                    foiministryrequestid,
                    foirequest_id,
                    foirequestversion_id,
                    version,
                    duedate,
                    description,
                    axisrequestid,

                    --*,
                        ROW_NUMBER() OVER (
                            PARTITION BY foiministryrequestid 
                            ORDER BY created_at DESC
                        ) AS rn
                    FROM foi_mod.foiministryrequests
                    where foiministryrequestid in ({comma_list})
                ) sub
                WHERE rn = 1) sq

        join 

        (select deliverymodeid, foirequestid, requesttype, version
        from foi_mod.foirequests 
        ) r on r.foirequestid = sq.foirequest_id and r.version = sq.foirequestversion_id

        join

        (SELECT *
                FROM (
                    SELECT 
                    created_at as firstcreated_at,
                    createdby,
                    foiministryrequestid,
                    --*,
                        ROW_NUMBER() OVER (
                            PARTITION BY foiministryrequestid 
                            ORDER BY created_at asc
                        ) AS rn
                    FROM foi_mod.foiministryrequests
                ) sub
                WHERE rn = 1) sq2 on sq2.foiministryrequestid = sq.foiministryrequestid


        left join 
        (select description as subject, foiministryrequestid, foiministryrequestversion from foi_mod.subjectcodes
        join foi_mod.foiministryrequestsubjectcodes using (subjectcodeid)) sq3 on sq.foiministryrequestid = sq3.foiministryrequestid 
        and sq.version = sq3.foiministryrequestversion
        """

    print(query)

    df = spark.sql(query)
    print(df.count())
    df.show()

    if not df.isEmpty():
        getdays = udf(getbusinessdaysbetween, IntegerType())

        print(duedate_value)
        print(type(duedate_value))

        df = df.withColumn("overduedays", getdays(lit(date.today()), df['duedate']))
        df = df.withColumn("elapseddays", getdays(lit(date.today()), df['firstcreated_at']))
        df = df.withColumn("passduedays", getdays(lit(date.today()), df['duedate']))
        df = df.withColumn("remainingdays", getdays(df['duedate'], lit(date.today())))

        df.show(df.count(), truncate=False)



        # order of columns here is important!
        df_mapped = df.selectExpr(
            "NULL as actionid",        
            "foirequest_id as foirequestid",
            f"{runcycleid} as runcycleid",
            "NULL as actiontype",
            "description as description",
            "'' as priority",
            "'' as emailaddress",
            "firstcreated_at as createddate",
            "NULL as actiondate",
            "duedate as duedate",
            "'' AS responsedate",
            "'' AS parentactionid",
            "createdby AS createdby",
            "subject AS subject",
            "'' as programofficeid",
            "'' AS reqfordocstatusid",
            "completeddate AS completeddate",
            "NULL AS requestofficeid",
            "axisrequestid as visiblerequestid",
            "description AS requestdescription",
            "NULL as officeid",
            "requesttype as requesttypeid",
            "overduedays as overduedays",
            "elapseddays as elapseddays",
            "passduedays as passduedays",
            "NULL as rfdage",
            "remainingdays as remainingdays",
            "deliverymodeid as methodofdelivery",
            "'Y' AS activeflag",
            "'FOIMOD' as sourceoftruth"
        )
        df_mapped.show()
        df_mapped.write.format("delta").mode("append").option("mergeSchema", "false").insertInto("factrequestfordocuments")
    else:
        print("No results found for the query.")

    etl_helpers.end_run_cycle(runcycleid, 't', "factRequestForDocuments")
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', "factRequestForDocuments", "Credentials not available")
except Exception as e:    
    if (str(e) == "no changes for today"):
        # print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', "factRequestForDocuments")
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', "factRequestForDocuments", f"An error occurred: {e}")